# Performance Benchmarking

**Yuning Zhang, TU Delft, 2021.10.27**

In [ ]:
using Random

### Allocate a large sequence of random locations 

In [ ]:
function _rand_loc_spherical(r_max=1.0::Real, r_min=0.0::Real; projection=:false)
    r=rand()*(r_max-r_min)+r_min
    ϕ=rand()*2pi
    if projection
        θ=pi/2
    else
        θ=rand()*pi
    end
    [r*sin(θ)*cos(ϕ),r*sin(θ)*sin(ϕ),r*cos(θ)]
end

In [ ]:
@time collect(_rand_loc_spherical() for i in 1:10^6);

  0.122624 seconds (1.04 M allocations: 116.910 MiB, 19.80% compilation time)


In [ ]:
function rand_locs_spherical(r_max=1.0::Real, r_min=0.0::Real; N=1::Int, projection=:false)
    r=rand(N).*(r_max-r_min).+r_min
    ϕ=rand(N).*2pi
    if projection
        θ=ones(N).*pi/2
    else
        θ=rand(N).*pi
    end
    collect(zip(r.*sin.(θ).*cos.(ϕ),r.*sin.(θ).*sin.(ϕ),r.*cos.(θ)))
end

In [33]:
@time rand_locs_spherical(N=10^6);

  0.063351 seconds (20 allocations: 91.553 MiB, 2.87% gc time)


**Conclusion1, it's better to use vectorized approach**

In [ ]:
function rand_locs_spherical(r_max=1.0::Real, r_min=0.0::Real; N=1::Int, projection=:false)
    M=zeros()
    M[:,1]=rand(N).*(r_max-r_min).+r_min
    M[:,2]=rand(N).*2pi
    if projection
        M[:,3]=ones(N).*pi/2
    else
        θ=rand(N).*pi
    end
    collect(zip(r.*sin.(θ).*cos.(ϕ),r.*sin.(θ).*sin.(ϕ),r.*cos.(θ)))
end

In [6]:
function rand_locs_cubic(a::Real, b::Real; N=1, projection=:false)
    M=zeros(N,3)
    if projection
        for i in 1:N
            M[i,:]=rand([-1,1],3).*rand(3).*(b-a).+a
        end
    else
        for i in 1:N
            M[i,1:2]=rand([-1,1],2).*rand(2).*(b-a).+a
        end
    end
end

rand_locs_cubic (generic function with 1 method)

In [7]:
@time rand_locs_cubic(1,0,N=10^6,projection=:true)

  0.237117 seconds (4.00 M allocations: 434.891 MiB, 29.62% gc time)


In [8]:
@time rand_locs_cubic(1,0,N=10^6,projection=:false)

  0.240182 seconds (4.00 M allocations: 389.099 MiB, 30.89% gc time)


In [21]:
function rand_locs_cubic(a::Real, b::Real; N=1, projection=:false)
    M=zeros(N,3)
    rand!(M,[1,-1])
    M.*=rand(N,3).*(b-a).+a
    if projection
        M[:,end].=0.0
    end
    M
end

rand_locs_cubic (generic function with 1 method)

In [35]:
@time rand_locs_cubic(1,0,N=10^5);

  0.001701 seconds (5 allocations: 4.578 MiB)


In [34]:
@time rand_locs_cubic(1,0,N=10^6,projection=:true);

  0.028170 seconds (5 allocations: 45.777 MiB, 4.26% gc time)


**Conclusion2, Use less allocations to reduce garbage collection time**